In [ ]:
'''
Задача 4 (3 балла). Компания друзей собралась пойти в поход. Забот и затрат при 
подготовке похода оказалось много: кто-то закупал еду, кто-то брал в аренду снаряжение, 
кто-то заказывал транспорт. Когда всё было готово, друзья решили подсчитать, кто 
сколько денег потратил и, соответственно, кто кому сколько денег должен перевести. 
Статей расходов оказалось очень много, участников похода было тоже много, поэтому 
сделать все расчеты вручную оказалось затруднительно. 
Напишите программу, которая по информации о том, кто сколько денег потратил, 
определит: кто, кому и сколько денег должен перевести, чтобы расходы всех участников 
похода оказались одинаковыми (с точностью до копейки). Количество переводов при этом 
должно быть как можно меньше. 
Входные данные 
В первой строке через пробел записаны имена всех участников похода. Имена 
уникальны, каждое имя состоит из латинских букв. Длина каждого имени – не более 20 
символов, количество имен – не более 100. 
Во второй строке записано одно целое число N – количество покупок, которое было 
сделано при подготовке похода. 
Далее следует N строк, каждая из которых описывает одну покупку и содержит имя 
того, кто эту покупку оплачивал, и одно целое число – сумму покупки. Имя и число 
разделены пробелом. Гарантируется, что имя есть в общем списке участников похода. 
Выходные данные 
В первой строке выведите одно число M – минимальное количество переводов, 
которые нужно совершить. 
Далее выведите M строк, в каждой указав два имени и вещественное число через 
пробел: кто, кому и сколько должен перевести. Все суммы переводов должны быть 
округлены до 2 знаков после запятой. Если существует несколько вариантов переводов, то 
выведите любой из них – главное, чтобы их количество было минимальным. 

Пример 
Входные данные 
Ivan Aleksej Igor 
3 
Ivan 500 
Aleksej 100 
Ivan 200
Выходные данные  
2 
Aleksej Ivan 166.67 
Igor Ivan 266.67

Комментарий к примеру 
Иван потратил в сумме 700, но двое остальных переведут ему в сумме 433.34, итого 
затраты Ивана составят 266.66. Алексей потратил 100 и должен перевести Ивану 166.67, в 
итоге его затраты составят 266.67. Игорь ничего не потратил на покупки, и все его затраты – это перевод 
266.67 Ивану. Таким образом, затраты всех трех друзей окажутся 
одинаковыми с точностью до копейки (затраты Ивана на 1 копейку меньше, чем у 
остальных). Очевидно, что одним переводом добиться равных затрат не получится, 
поэтому 2 – это минимальное количество переводов. 
Заметим, что такой набор переводов – не единственно возможный. Например, Алексей 
мог бы перевести 166.67 Игорю, а Игорь перевести Ивану 433.34
'''

In [5]:
def main(expenses):
    # Подсчёт всей суммы
    sum_expenses = sum(expenses.values())
    # Подсчёт суммы на каждого человека
    average_expenses = round(sum_expenses / len(expenses), 2)
    # Подсчёт суммы остатка у одного из людей, если нельзя разделить на равных (например 1 копейка)
    remains = round(abs(sum_expenses - len(expenses)*average_expenses), 2)
    # Высчитаем из трат отдельных людей траты на каждого
    for name in expenses:
        expenses[name] = round(expenses[name]-average_expenses, 2)
    # Словари для положительных (кредиторы) и отрицательных (должники) значений
    debtors = {}
    creditors = {}
    for name, amount in expenses.items():
        if amount < 0:
            debtors[name] = amount
        elif amount > 0:
            creditors[name] = amount
    # Сортируем
    debtors = dict(sorted(debtors.items(), key=lambda item: item[1])) # По возрастанию
    creditors = dict(sorted(creditors.items(), key=lambda item: item[1], reverse=True)) # По убыванию

    # Вычисляем операции по переводам. Для нахождения минимального возможного количества операций
    # Мы сначала ищем можно ли какие то долги закрыть у должника и кридитора одновременно, если нет
    # уменьшаем показатели в зависимосити у кого больше сумма (кридитора или должника).
    transactions = [] # Список для транзакций
    for debtor_name, debtor_amount in debtors.items():
        while (debtor_amount !=0 and round(debtor_amount, 2) != round(remains, 2) and round(debtor_amount, 2) != -round(remains, 2)):
            # Проходим по спискам и ищем людей, которые могут закрыть друг друга с одного перевода
            flag = 1
            for name, amount in debtors.items():
                for creditors_name, creditors_amount in creditors.items():
                    if amount == -creditors_amount and amount != 0:
                        transactions.append(f"{name} {creditors_name} {abs(amount):.2f}")
                        debtors[name] = 0
                        amount = 0
                        creditors[creditors_name] = 0
                        creditors_amount = 0
                        if debtor_name == name: 
                            flag = 0
                            debtor_amount = 0
            # Начинаем закрывать долги
            if flag:
                for creditors_name, creditors_amount in creditors.items():
                    if creditors_amount > -debtor_amount:
                        if debtor_amount != 0:
                            transactions.append(f"{debtor_name} {creditors_name} {abs(debtor_amount):.2f}")
                            creditors[creditors_name] += round(debtor_amount, 2)
                            creditors_amount += round(debtor_amount, 2)
                            debtors[debtor_name] = 0
                            debtor_amount = 0
                    elif debtor_amount != 0:
                        transactions.append(f"{debtor_name} {creditors_name} {abs(creditors_amount):.2f}")
                        debtors[debtor_name] += round(creditors_amount, 2)
                        debtor_amount += round(creditors_amount, 2)
                        creditors[creditors_name] = 0
                        creditors_amount = 0
    return transactions

if __name__ == "__main__":
    # Получаем имена и записываем их в словарь со значением 0
    participants = input().split()
    expenses = {name: 0.0 for name in participants}
    # Получаем количество трат
    n = int(input())
    # Подсчитываем кто сколько потратил (используются словари)
    for i in range(n):
        name, amount = input().strip().split()
        amount = float(amount)
        expenses[name] += amount
    result = main(expenses)
    print (len(result))
    for i in result:
        print(i)

2
Igor Ivan 266.67
Aleksej Ivan 166.66


In [ ]:
'''
Тесты: 
1 тест)
Вход:
Ivan Aleksej Igor 
3 
Ivan 500 
Aleksej 100 
Ivan 200 
Выход:
2
Igor Ivan 266.67
Aleksej Ivan 166.66

2 тест) 
Вход:
Ivan Aleksej Igor Vova
3
Ivan 300 
Aleksej 100 
Vova 200 
Выход:
2
Igor Ivan 150.00
Aleksej Vova 50.00

3 тест) 
Вход:
Ivan Aleksej Igor Vova Alina
3
Ivan 240 
Aleksej 100 
Vova 260
Выход:
3
Igor Ivan 120.00
Alina Vova 120.00
Aleksej Vova 20.00

4 тест)
Вход:
Ivan Aleksej Igor 
3 
Ivan 355 
Aleksej 268 
Igor 10.5
Выход:
2
Igor Ivan 143.83
Igor Aleksej 56.83
'''